<a href="https://colab.research.google.com/github/RayBasilio123/IC_2022/blob/main/hyperForest_Porto_Alegre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score
from datetime import datetime
from matplotlib import pyplot
from sklearn.ensemble import RandomForestRegressor
from math import nan
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
def get_x2(df,lista,lags,Eto,lags_eto):
  lags=lags
  lista_aux=[]
  lista_aux2=[]
  max_lag=0;
  data = pd.DataFrame()
  eta_nois = pd.DataFrame()
  eta_nois['Data']=df['Data']
  for coluna in lista:
    data[coluna] = df[coluna]       

  for i in range(len(lags)):
    for j in range(len(lags[i])):
      lista_aux = data.iloc[:,i].tolist()
      for displacement in range((lags[i][j])):
        if max_lag<(lags[i][j]):
          max_lag=(lags[i][j])
        del lista_aux[len(lista_aux)-1]
        lista_aux.insert(0,nan)
      eta_nois[((data.iloc[:,i]).name)+("_t-")+str(lags[i][j])]=(lista_aux)  
  
  for i in range(len(lags_eto)):
    lista_aux2=df[Eto].iloc[:,0].tolist()
    for displacement in range((lags_eto[i])):
      if max_lag<(lags_eto[i]):
        max_lag=(lags_eto[i])
      del lista_aux2[len(lista_aux2)-1]
      lista_aux2.insert(0,nan)
    eta_nois[((df[Eto].iloc[:,0]).name)+("_t-")+str(lags_eto[i])]=lista_aux2        
  return eta_nois,max_lag,lista,lags,Eto,lags_eto;

In [3]:
def get_x30(df,lista, Eto):
  ix = []
  idx =  [i for i in np.arange(1, 31)]
  for i in range(len(lista)):
    ix.append(idx)

  resultado = get_x2(df, lista, ix, Eto, idx)
  return resultado

In [4]:
def column_Filter(df,dias):
  list_column = []
  # pegando colunas do data frame
  for column in (df.columns):
    if column != 'Data':
      list_column.append(column)
  # prosseguindo com a filtração
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
 
  # filtra e  remove colunas 
  for i in range(len(list_column)):
    j=list_column[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  # print("Lista_Filtrada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  # print("lista",lista)
  # print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  # print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  # print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3









In [5]:
def resource_ranking(df,lista_filtrada3,variavel_alvo) :
  
  # resource_ranking(df_la,lista_filtrada2,"temp_min")  
  #Monta a tabela com os dados filtrados 
  
  tab=get_x2(df,lista_filtrada3[0],lista_filtrada3[1],lista_filtrada3[2],lista_filtrada3[3])
 
  # dff=tab[0].drop("Data",axis=1)
  
  # train_size = int(((len(tab[0]))-tab[1]) )
  # dff=dff.iloc[tab[1]:train_size,:]

  # array1 = dff.values

  # df= df.drop("Data",axis=1)
  # df=df.iloc[tab[1]:train_size,:]
  # array2 = df[variavel_alvo]
  selecao_treino = (tab[0]['Data'] >= '2014') & (tab[0]['Data'] <= '2018-12-31')
  
  dff=tab[0][selecao_treino].drop("Data",axis=1)

  train_size = int(((len(tab[0][selecao_treino]))-tab[1]) )
  dff=dff.iloc[tab[1]:train_size,:]
  array1 = dff.values

  df= df[selecao_treino].drop("Data",axis=1)
  df=df.iloc[tab[1]:train_size,:]
  array2 = df[variavel_alvo]

 

  X =array1[:,0:len(tab[0].columns)]
  Y = array2
  # feature extraction
  test = SelectKBest(score_func=f_regression, k=4)
  fit = test.fit(X, Y)
  # summarize scores
  set_printoptions(precision=3)
 

    
  # print("Selecao_univariada",fit.scores_)
  f=fit.scores_
  f_ord = sorted(f,reverse=True)
  # print("Selecao_univariada_ordenada",f_ord)

  ll=[]
  for y in range(len(f_ord)):
    for i in range(len(f)):
      if (f_ord[y]==f[i]):
        ll.append(i)
        # print("f[i]",f[i])
        # print("f_ord[y]",f_ord[y])
  leg_seq=[]
  for i in range(len(ll)):
    leg_seq.append(dff.columns[ll[i]])    

  # print("Colunas_selecionadas [0]- Selecao_univariada",leg_seq)
 
  model = ExtraTreesRegressor(n_estimators=10,random_state=42)
  model.fit(X,Y)
  g=model.feature_importances_
  g_ord=sorted(model.feature_importances_,reverse=True)
  # print("Importancia",g)
  # print("Importancia_ordenada",g_ord)
  jj=[]
  for y in range(len(g_ord)):
    for i in range(len(g)):
      if (g_ord[y]==g[i]):
        jj.append(i)
  leg_seq2=[]
  for i in range(len(ll)):
    leg_seq2.append(dff.columns[jj[i]])    
  # print("Colunas_selecionadas [1]- Importância do recurso",leg_seq2)
  return leg_seq,leg_seq2

In [6]:


def list_Format(melhores_recursos,q,dias):
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
  q = len(melhores_recursos) if q==0 else q
  # filtra e  remove colunas 
  for i in range(q):
    j=melhores_recursos[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  print("lista_selecionada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  print("lista",lista)
  print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3

In [7]:
def train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo,data_Itest= '2019-01-01',data_Ftest= '2020-12-31',data_Itreino='2014-01-01',data_Ftreino='2018-12-31'):
  tabela = get_x2(df, lista,lags,Eto,lags_eto)
  selecao_treino = (tabela[0]['Data'] >= data_Itreino) & (tabela[0]['Data'] <= data_Ftreino)
  selecao_teste = (tabela[0]['Data'] >= data_Itest) & (tabela[0]['Data'] <= data_Ftest)

  x1_train = tabela[0][selecao_treino].drop("Data", axis=1)
  x1_test = tabela[0][selecao_teste].drop("Data", axis=1)
  
  y1_train = df[variavel_Alvo][selecao_treino]
  y1_test = df[variavel_Alvo][selecao_teste]

  x1_train = x1_train[tabela[1]:]
  y1_train = y1_train[tabela[1]:]

  
  print("***************************************")
  print(x1_train.shape,"x1_train")
  #[6569 rows x 10 columns] x1_train
  print("*****************++*********************")
  print(x1_test.shape,"x1_test")
  print("******************---********************")
  print(y1_train.shape,"y1_train")
  print("****************1234*********************")
  print(y1_test.shape,"y1_test")
  print("*****************asddfg******************")
 
  
  return x1_train, x1_test,y1_train, y1_test

In [8]:
def gridSearchF(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  param_grid = {
      'bootstrap': [True],
      'max_depth': [10,80, 90, 100, 110],
      'max_features': [2, 4,5],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [2,8, 10, 12],
      'n_estimators': [100,300, 1000]
  }

  # Create a base model
  rf = RandomForestRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

  print(" -----------------grid_search.fit--------------")
 
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  
  
    
  return grid_search.best_params_

def gridSearchFs(df,parametros,variavel_Alvo):

  lista_colunas=["gridSearch"]
  tb = pd.DataFrame(columns=lista_colunas)
  
  print("gridSearchFs")
  print('---------------------------------------------------------------------------------')
  
  for x in range(len(parametros)):
    a=gridSearchF(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
    
    print(x,"Best_hyperparameter_Arvore", a)
   
  print('---------------------------------------------------------------------------------')
  

  return a




In [9]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)

In [10]:
url=('https://raw.githubusercontent.com/RayBasilio123/database/main/INMET/CSV/J_Database/PORTO_ALEGRE_J_2014_2020.csv')
df = pd.read_csv(url, sep=",", encoding = "ISO-8859-1")
df=df.drop(columns=['Unnamed: 0'])

In [11]:
df

,Data,vento,radiacao,temp_max,temp_min,umi_max,umi_min,umi_rel,press_atm,temp_media,Eto,J
0,2014-01-01,0.928706,1.371140,34.1,23.5,91.0,54.0,73.666667,1001.320833,27.437500,2.917568,1
1,2014-01-02,1.446039,0.778367,34.4,24.7,90.0,50.0,73.125000,996.962500,28.035417,3.648835,2
2,2014-01-03,1.031549,0.193829,27.9,19.4,94.0,72.0,88.708333,1003.116667,22.054167,0.982221,3
3,2014-01-04,1.305798,1.176481,28.7,17.4,84.0,34.0,62.875000,1012.112500,22.343750,3.564776,4
4,2014-01-05,1.034666,1.337948,32.5,17.6,86.0,28.0,62.583333,1010.945833,24.377083,3.854867,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2020-12-27,1.433573,0.738310,31.0,21.4,90.0,49.0,74.750000,1008.025000,24.762500,2.926815,362
2553,2020-12-28,1.823131,1.169640,30.6,21.0,91.0,52.0,72.875000,1008.158333,24.950000,3.756797,363
2554,2020-12-29,1.168674,1.190469,33.5,20.5,92.0,41.0,70.250000,1003.887500,25.910417,3.679275,364
2555,2020-12-30,1.181139,1.043798,34.9,21.4,92.0,45.0,72.666667,999.675000,26.964583,3.546235,365


In [12]:
atributeP= [ 'vento'	,'radiacao',	'temp_max',	'temp_min',	'umi_max',	'umi_min',	'umi_rel',	'press_atm',	'temp_media','J']
resultadoP=get_x30(df,atributeP,["Eto"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [13]:
for x in [1,3,7,10]:
 lista_Filtrada=column_Filter(resultadoP[0],x)
 ranking_data = resource_ranking(df,lista_Filtrada,'Eto')
 lista_Formatada1=list_Format(ranking_data[0],10,0)
 lista_Formatada2=list_Format(ranking_data[1],10,0)
 print('\n -----------Para----' + str(x) + '----dias --------\n')
 print("lista_Formatada_1 -------> ",lista_Formatada1)
 print("lista_Formatada_2 -------> ",lista_Formatada2)
 print("lista_Formatada_3 -------> ",lista_Filtrada)
 print('\n ---------------------------------------------\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['Eto', '1'], ['radiacao', '1'], ['Eto', '2'], ['temp_max', '1'], ['temp_media', '1'], ['Eto', '3'], ['temp_media', '5'], ['temp_media', '2'], ['temp_media', '4'], ['temp_media', '3']]
lista ['radiacao', 'temp_max', 'temp_media']
Eto ['Eto']
lags [[1], [1], [1, 5, 2, 4, 3]]
lags_eto [1, 2, 3]
lista_selecionada [['Eto', '1'], ['radiacao', '1'], ['temp_media', '4'], ['Eto', '2'], ['radiacao', '3'], ['temp_min', '3'], ['temp_media', '13'], ['umi_min', '1'], ['umi_rel', '1'], ['temp_max', '1']]
lista ['radiacao', 'temp_media', 'temp_min', 'umi_min', 'umi_rel', 'temp_max']
Eto ['Eto']
lags [[1, 3], [4, 13], [3], [1], [1], [1]]
lags_eto [1, 2]

 -----------Para----1----dias --------

lista_Formatada_1 ------->  [['radiacao', 'temp_max', 'temp_media'], [[1], [1], [1, 5, 2, 4, 3]], ['Eto'], [1, 2, 3]]
lista_Formatada_2 ------->  [['radiacao', 'temp_media', 'temp_min', 'umi_min', 'umi_rel', 'temp_max'], [[1, 3], [4, 13], [3], [1], [1], [1]], ['Eto'], [1, 2]]
lista_Formatada_3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['Eto', '3'], ['temp_media', '5'], ['temp_media', '4'], ['temp_media', '3'], ['temp_media', '6'], ['Eto', '4'], ['temp_media', '7'], ['temp_media', '12'], ['Eto', '5'], ['temp_media', '11']]
lista ['temp_media']
Eto ['Eto']
lags [[5, 4, 3, 6, 7, 12, 11]]
lags_eto [3, 4, 5]
lista_selecionada [['temp_media', '7'], ['radiacao', '3'], ['Eto', '3'], ['temp_media', '3'], ['J', '26'], ['temp_media', '13'], ['temp_max', '3'], ['radiacao', '6'], ['Eto', '5'], ['J', '12']]
lista ['temp_media', 'radiacao', 'J', 'temp_max']
Eto ['Eto']
lags [[7, 3, 13], [3, 6], [26, 12], [3]]
lags_eto [3, 5]

 -----------Para----3----dias --------

lista_Formatada_1 ------->  [['temp_media'], [[5, 4, 3, 6, 7, 12, 11]], ['Eto'], [3, 4, 5]]
lista_Formatada_2 ------->  [['temp_media', 'radiacao', 'J', 'temp_max'], [[7, 3, 13], [3, 6], [26, 12], [3]], ['Eto'], [3, 5]]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel', 'press_atm', 'temp_media'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['temp_media', '7'], ['temp_media', '12'], ['temp_media', '11'], ['temp_media', '8'], ['temp_media', '13'], ['temp_media', '10'], ['temp_media', '9'], ['Eto', '7'], ['Eto', '13'], ['temp_max', '7']]
lista ['temp_media', 'temp_max']
Eto ['Eto']
lags [[7, 12, 11, 8, 13, 10, 9], [7]]
lags_eto [7, 13]
lista_selecionada [['J', '23'], ['temp_media', '12'], ['temp_media', '7'], ['J', '14'], ['temp_media', '11'], ['temp_media', '16'], ['J', '22'], ['temp_media', '13'], ['J', '19'], ['radiacao', '13']]
lista ['J', 'temp_media', 'radiacao']
Eto []
lags [[23, 14, 22, 19], [12, 7, 11, 16, 13], [13]]
lags_eto []

 -----------Para----7----dias --------

lista_Formatada_1 ------->  [['temp_media', 'temp_max'], [[7, 12, 11, 8, 13, 10, 9], [7]], ['Eto'], [7, 13]]
lista_Formatada_2 ------->  [['J', 'temp_media', 'radiacao'], [[23, 14, 22, 19], [12, 7, 11, 16, 13], [13]], [], []]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel',

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


lista_selecionada [['temp_media', '12'], ['temp_media', '11'], ['temp_media', '13'], ['temp_media', '10'], ['Eto', '13'], ['temp_min', '12'], ['temp_media', '14'], ['Eto', '12'], ['Eto', '10'], ['temp_min', '11']]
lista ['temp_media', 'temp_min']
Eto ['Eto']
lags [[12, 11, 13, 10, 14], [12, 11]]
lags_eto [13, 12, 10]
lista_selecionada [['temp_media', '11'], ['J', '30'], ['J', '28'], ['J', '24'], ['temp_media', '12'], ['temp_media', '16'], ['temp_media', '15'], ['J', '18'], ['J', '14'], ['J', '20']]
lista ['temp_media', 'J']
Eto []
lags [[11, 12, 16, 15], [30, 28, 24, 18, 14, 20]]
lags_eto []

 -----------Para----10----dias --------

lista_Formatada_1 ------->  [['temp_media', 'temp_min'], [[12, 11, 13, 10, 14], [12, 11]], ['Eto'], [13, 12, 10]]
lista_Formatada_2 ------->  [['temp_media', 'J'], [[11, 12, 16, 15], [30, 28, 24, 18, 14, 20]], [], []]
lista_Formatada_3 ------->  [['vento', 'radiacao', 'temp_max', 'temp_min', 'umi_max', 'umi_min', 'umi_rel', 'press_atm', 'temp_media', 'J'], 

# Para  1 dia - Grid Forest


In [15]:
lista_Formatada_1=  [['radiacao', 'temp_max', 'temp_media'], [[1], [1], [1, 5, 2, 4, 3]], ['Eto'], [1, 2, 3]]
lista_Formatada_2=  [['radiacao', 'temp_media', 'temp_min', 'umi_min', 'umi_rel', 'temp_max'], [[1, 3], [4, 13], [3], [1], [1], [1]], ['Eto'], [1, 2]]

In [16]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")

gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1821, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1821,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
***************************************
(1813, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1813,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi

#Para  3 dias - Grid Forest

In [17]:
lista_Formatada_1=  [['temp_media'], [[5, 4, 3, 6, 7, 12, 11]], ['Eto'], [3, 4, 5]]
lista_Formatada_2=  [['temp_media', 'radiacao', 'J', 'temp_max'], [[7, 3, 13], [3, 6], [26, 12], [3]], ['Eto'], [3, 5]]

In [18]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1814, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1814,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
***************************************
(1800, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1800,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candid

# Para 7 dias Grid Forest

In [19]:

lista_Formatada_1=  [['temp_media', 'temp_max'], [[7, 12, 11, 8, 13, 10, 9], [7]], ['Eto'], [7, 13]]
lista_Formatada_2=  [['J', 'temp_media', 'radiacao'], [[23, 14, 22, 19], [12, 7, 11, 16, 13], [13]], [], []]

In [20]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1813, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1813,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 100}
***************************************
(1803, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1803,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candi

# Para 10 dias - Grid Forest

In [21]:
lista_Formatada_1=  [['temp_media', 'temp_min'], [[12, 11, 13, 10, 14], [12, 11]], ['Eto'], [13, 12, 10]]
lista_Formatada_2=  [['temp_media', 'J'], [[11, 12, 16, 15], [30, 28, 24, 18, 14, 20]], [], []]

In [22]:
parametrosSearchA=gridSearchFs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


gridSearchFs
---------------------------------------------------------------------------------
***************************************
(1812, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1812,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
0 Best_hyperparameter_Arvore {'bootstrap': True, 'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
***************************************
(1796, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1796,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 540 candid